In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from itertools import product
from requests import ConnectionError

#from requests.auth import HTTPDigestAuth

In [ ]:
df = None

In [ ]:
#set authorizations

from requests.auth import HTTPBasicAuth

s = requests.Session()
s.auth = ('energydept', 'Independusa*')
#s.headers['User-Agent'] = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36"
s.headers['Referer'] = "https://www.dmr.nd.gov/oilgas/feeservices/stateprod.asp"
s.headers['Host'] = "www.dmr.nd.gov"
s.headers['Accept'] = "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8"
s.headers['Accept-Language'] = "en-US,en;q=0.8"
s.headers['Accept-Encoding'] = 'gzip, deflate, br'
s.headers['Upgrade-Insecure-Requests'] = "1"
s.headers['Content-Type'] = "application/x-www-form-urlencoded"
response = s.get('https://www.dmr.nd.gov/oilgas/feeservices/stateprod.asp', auth=HTTPBasicAuth('energydept', 'Independusa*'), verify=False)


In [ ]:
#run historic years which have all 12 months of data available (range 1-13)
#add 1- range works so that it values not including last
#range(2010, 2018) will pull all years 2010-2017
for month_i, year in product(range(1,13), range(2010, 2018)):
    time.sleep(7)
    try:
        month = str("%02d" % (month_i))

        payload = "VTI-GROUP=0&SELECTMONTH="+month+"&SELECTYEAR="+str(year)+"&B1=Get+State+Volumes"
        response = s.post('https://www.dmr.nd.gov/oilgas/feeservices/stateprod.asp',data=payload,  verify=False)

        soup = BeautifulSoup(response.text, 'lxml')
        table = soup.find("table", {"id": "largeTableOutput"})
        data = []
        table_body = table.find('tbody')

        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele]) # Get rid of empty values


        columns = table.find_all('th')
        col_names = []
        for column in columns:
            col_names.append(column.text.strip())
        col_names

        if df is None:
            df=pd.DataFrame(data,columns=col_names)
        else:
            df_temp=pd.DataFrame(data,columns=col_names)
            df = df.append(df_temp)
    except: 
        try:
    
            month = str("%02d" % (month_i))

            payload = "VTI-GROUP=0&SELECTMONTH="+month+"&SELECTYEAR="+str(year)+"&B1=Get+State+Volumes"
            response = s.post('https://www.dmr.nd.gov/oilgas/feeservices/stateprod.asp',data=payload,  verify=False)

            soup = BeautifulSoup(response.text, 'lxml')
            table = soup.find("table", {"id": "largeTableOutput"})
            data = []
            table_body = table.find('tbody')

            rows = table_body.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                cols = [ele.text.strip() for ele in cols]
                data.append([ele for ele in cols if ele]) # Get rid of empty values


            columns = table.find_all('th')
            col_names = []
            for column in columns:
                col_names.append(column.text.strip())
            col_names

            if df is None:
                df=pd.DataFrame(data,columns=col_names)
            else:
                df_temp=pd.DataFrame(data,columns=col_names)
                df = df.append(df_temp)
        except OSError as err:
            print("OS error: {0}".format(err))
            continue
        except ValueError:
            print("Could not convert data to an integer.")
            continue
        except:
            print("Unexpected error:", sys.exc_info()[0])
            continue

In [ ]:
#convert date
df['Date1'] =  pd.to_datetime(df['Date'], format='%m-%Y')

In [ ]:
#write to csv
df.to_csv('Z:/OilProduction/ND/ND_website_scrape/Current_Website_Output/ND_well_2010_2017.csv', sep=',', index=False)

In [ ]:
#run 2018 which does not have all months of data available since it is current month
#running through June/2018
for month_i, year in product(range(1,6),range(2018, 2019)):
    time.sleep(7)
    try:
        month = str("%02d" % (month_i))

        payload = "VTI-GROUP=0&SELECTMONTH="+month+"&SELECTYEAR="+str(year)+"&B1=Get+State+Volumes"
        response = s.post('https://www.dmr.nd.gov/oilgas/feeservices/stateprod.asp', data=payload,  verify=False)

        soup = BeautifulSoup(response.text, 'lxml')
        table = soup.find("table", {"id": "largeTableOutput"})
        data = []
        table_body = table.find('tbody')

        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele]) # Get rid of empty values


        columns = table.find_all('th')
        col_names = []
        for column in columns:
            col_names.append(column.text.strip())
        col_names

        if df is None:
            df=pd.DataFrame(data,columns=col_names)
        else:
            df_temp=pd.DataFrame(data,columns=col_names)
            df = df.append(df_temp)
    except OSError as err:
        print("OS error: {0}".format(err))
        continue
    except ValueError:
        print("Could not convert data to an integer.")
        continue
    except:
        print("Unexpected error:", sys.exc_info()[0])
        continue


                  



In [ ]:
#convert date
df['Date1'] =  pd.to_datetime(df['Date'], format='%m-%Y')

In [ ]:
#write to csv
df.to_csv('Z:/OilProduction/ND/ND_website_scrape/Current_Website_Output/ND_well_prod_2018.csv', sep=',', index=False)